<a href="https://colab.research.google.com/github/AntonioDeFA/etl_pyspark/blob/develop/Dados_IBGE_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark findspark

In [2]:
import findspark
import unicodedata
import seaborn as sns
import matplotlib.pyplot as plt
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, Row
from pyspark.sql.window import Window
from pyspark.sql.functions import col

In [3]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('ASCII')

In [4]:
findspark.init()

spark = SparkSession.builder \
    .appName("Dados IBGE") \
    .getOrCreate()

input_path_ibge = "/content/cities-brazil-ibge.csv"
data_ibge = spark.read.csv(input_path_ibge,
                           header=True,
                           inferSchema=True,
                           sep=",",
                           encoding="UTF-8")

input_path_covid = "/content/cases-brazil-cities-time_2022.csv"
data_covid = spark.read.csv(input_path_covid,
                            header=True,
                            inferSchema=True,
                            sep=",",
                            encoding="UTF-8")

In [21]:
windowSpec = Window.partitionBy("city").orderBy(col("date").desc())

last_cases_covid_data = data_covid.withColumn("row_num",
                                       row_number().
                                       over(windowSpec)).\
                                       filter(col("row_num") == 1).\
                                       drop("row_num")


In [26]:

filtered_data = last_cases_covid_data.filter(
    (col("state") == "PB") &
    upper(col("city")).startswith("CO") &
     (((col("deaths")/col("totalCases"))*100)>0)
).select("*")